# Preparation

Intall prerequisites for YOLOX and check if you're using CPU or GPU.

In [ ]:
# Clone the repo and install modules
!git clone https://github.com/Megvii-BaseDetection/YOLOX
%cd YOLOX
!pip install -qr requirements.txt
!pip install -q -v -e .
!pip install -q cython
!pip install -q 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'

In [ ]:
import torch
print('Torch device : %s' % (torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'No GPU aviable'))

# Download and prepare dataset

Download your VOC datset to your colab environment.

You can make your own dataset using Roboflow online tool : https://app.roboflow.com

Export it (Pascal VOC type) and paste the link in the following cell :

In [ ]:
# Skip this cell if you don't have a ready dataset
!curl -L "https://app.roboflow.com/ds/XXXXXXXXXXXXXXXXXXXXXXXXX" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

For this example, we'll be using Aquarium Dataset made by Roboflow : https://public.roboflow.com/object-detection/aquarium , downloaded directly from this github repository

In [ ]:
# Only necessary for the example
# Do not launch this cell if you use your own dataset
%cd /content
!mkdir YOLOX/train
!mkdir YOLOX/valid
!git clone https://github.com/NathanOD/YOLOX-Train
!cp -rf /content/YOLOX-Train/train/*.jpg /content/YOLOX/train
!cp -rf /content/YOLOX-Train/train/*.xml /content/YOLOX/train
!cp -rf /content/YOLOX-Train/valid/*.jpg /content/YOLOX/valid
!cp -rf /content/YOLOX-Train/valid/*.xml /content/YOLOX/valid

In [ ]:
# Convert your VOC dataset to COCO format
%cd /content/YOLOX
!git clone https://github.com/Kazuhito00/convert_voc_to_coco.git
!python convert_voc_to_coco/convert_voc_to_coco.py \
    train train/instances_train2017.json \
    --start_image_id=0
!python convert_voc_to_coco/convert_voc_to_coco.py \
    valid validation/instances_val2017.json \
    --start_image_id=10000000

In [5]:
!mkdir datasets/COCO
!mkdir datasets/COCO/train2017
!mkdir datasets/COCO/val2017
!mkdir datasets/COCO/annotations

!cp -rf train/*.jpg datasets/COCO/train2017
!cp -rf valid/*.jpg datasets/COCO/val2017
!cp -rf train/instances_train2017.json datasets/COCO/annotations
!cp -rf validation/instances_val2017.json datasets/COCO/annotations

Now you have to modify your config file to set some parameters like `num_classes` or `max_epochs`.

In [6]:
# config modification to train the model
import os
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [7]:
%%writetemplate /content/YOLOX/exps/default/yolox_m.py

import os

from yolox.exp import Exp as MyExp


class Exp(MyExp):
    def __init__(self):
        super(Exp, self).__init__()
        self.depth = 0.67
        self.width = 0.75
        self.num_classes = 7
        self.max_epoch = 40
        self.basic_lr_per_img = 0.01 / 64.0
        self.exp_name = os.path.split(os.path.realpath(__file__))[1].split(".")[0]

# Download Weights

These weights are pretrained on COCO dataset.

In [ ]:
%cd /content/YOLOX/
!wget https://github.com/Megvii-BaseDetection/YOLOX/releases/download/0.1.1rc0/yolox_m.pth

# Training

In [ ]:
# Install wandb if you want to track training results
!pip install -q wandb
import wandb
wandb.login()

You may have ton change batch size `-b` if you're trainig another YOLOX model.

In [ ]:
!python tools/train.py \
    -f exps/default/yolox_m.py \
    -d 1 \
    -b 16 \
    --fp16 \
    -o \
    -c yolox_m.pth

# Testing weights

Now it's time to check if your weights trained well !

In [ ]:
!python tools/demo.py image \
    -f exps/default/yolox_m.py \
    -c /content/YOLOX/YOLOX_outputs/yolox_m/best_ckpt.pth \
    --path /path/to/your/image \
    --conf 0.25 \
    --nms 0.45 \
    --tsize 640 \
    --save_result \
    --device gpu